In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import re
from openpyxl import Workbook, styles
from openpyxl.styles import PatternFill, Font
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.formatting.rule import Rule

def csv_files(path):
    """Função usada para ler os ficheiros que tem o simbolo do Euro"""
    import csv
    f = open(path, encoding='windows-1252', errors='ignore')
    data = []
    for row in csv.reader(f, delimiter=','):
        data.append(row)
    col = [*data[0]]
    data.pop(0)
    df = pd.DataFrame(data, columns=col)
    return df, col

def read_files(path, sheetname, n_skiprows, n_skip_columns, site_index):
    """
    Params:\n
    path: parth of file in the computer.\n
    n_skiprows: Number of rows to delete in the original file,.\n
    columns_to_convert: Columns to convert the data general type. \n
    n_skipcolumn: Columns to skip in the original file. \n
    endrow = pass 0 to read everything, 1 to count entire
    columns_order: List of columns names in specific order to pass in the engine.\n
    """
    df = pd.read_excel(path, sheet_name = sheetname, skiprows = n_skiprows)

    df = df.iloc[:,n_skip_columns:]

    # define the entiry columns which doesn't have NaN 
    df = df.dropna(subset=[site_index], axis=0)
    """cont = 0
    for i in df.iloc[:,site_col]:
        if pd.isnull(i):
            break # acertar o break
        else:
            cont +=1
    df = df.iloc[:cont, :]
    #df.columns = columns_order"""
    
    # convert intery columns to integer 
    #df[columns_integer_convert] = df[columns_integer_convert].fillna(0)
    #df[columns_integer_convert] = df[columns_integer_convert].astype('int64')

    return df

def lower_str(columns):
    newlist = list(map(lambda x: x.lower(), columns))
    return newlist

def check_df(df, error_msg):
    if df == None or df.empty:
        return f'{error_msg}'
    else:
        return df

def count_duplicates(lista):
    count_dict = {}
    for entry in lista:
        if entry in count_dict.keys():
            count_dict[entry] += 1
        else:
            count_dict[entry] = 1
    
    duplicates = {}
    for k, v in count_dict.items():
        if v > 1:
            duplicates[k] = v
    return pd.DataFrame.from_dict(duplicates, orient='index', columns=['# of Duplicates'])

def defining_df(df, column_range, number_col):
    cont = 0
    for i in df.iloc[:,number_col]:
        if pd.isnull(i):
            break # acertar o break
        else:
            cont +=1
    df = df.iloc[0:cont, :]
    return df

#old version
def check_columns(table, output_columns):
    """
    Check the total of number of missing columns and the missing columns in passed table.\n

    Params:\n
    table: contain the columns to be check.\n
    output_columns: columns structure at the final file. 

    Returns:\n
    Number of missing columns and a list that contains the name os missing columns.
    """
    total_received = len(table.columns)
    number_missing_columns = 0
    missing_columns = []
    #Counting of missing columns       
    #if country in contries:      
    for columns in output_columns:
        if columns.lower() not in [labels.lower() for labels in table]:
            number_missing_columns +=1
            missing_columns.append(columns)
    
    return total_received, number_missing_columns, missing_columns

def check_columns_received(df, bill_cols):
    twdb_col = lower_str(list(df.columns))
    col_miss = [i for i in bill_cols if i not in twdb_col]
    """
    for i in bill_cols:
        if i not in twdb_col:
            col_miss.append(i)"""
    df_col_missing = pd.DataFrame(col_miss, columns=['Column(s) Missing'], index=range(len(col_miss)))
    return df_col_missing

def replace_values(df, columns, value=""):
    """
    Está voltando para float
    """
    invalid_values = ['N/A', 'n/a',"0", 0, '-', '_', np.nan,'nan']
    #lista = []

    df.fillna(0, inplace=True)
    #df[column] = df[column].astype('int64')

    for column in columns:
        lista = []
        for index in df[column]:
            #print(f"{column} -> {index}")
            if index in invalid_values:
                lista.append(value)
            else:
                lista.append(index)
        df[column] = lista

    return df
      
def date_parser(df, columns, format=1, type_dates='normal'):
    t_col = type_dates.lower()
    if format == 1:
        type_date = "%d/%m/%Y"
    else:
        type_date = "%d-%m-%Y"
    for column in columns:
        if t_col == 'mixed':
            df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) \
                        and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
            df[column] = df[column].astype(str)
        else:
            df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
            df[column] = df[column].astype(str)

# Refactorar esse codigo para receber todas as colunas num dic
# Sendo as keys=columns e values= picklist for each column
def check_date_columns(df, df_index,status_col,columns, format):
    """
    Paramns \n
        Dates needs to be in string format not in datetime, otherwise raise an error.\n
        Convert entire column to string before.
    """
    df_dates = df[columns]
    df_dates['sites'] = df_dates[df_index]
    df_dates = df_dates.set_index('sites')

    df_de = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df_dates[df_index])
    if not df_duplicates.empty:
        df_dates.drop_duplicates(subset=[df_index], inplace=True)

    filtered = df[[df_index, status_col]]

    if format == 1:
        date_format = re.compile(r'\d{2}\-\d{2}\-\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                #print(type(df_dates.loc[de_site,de_column]))
                value = df_dates.loc[de_site,de_column]
                if date_format.match(value) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                    else:
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            df_de.reset_index()
            df_de = pd.merge(df_de, filtered, how='left', on=df_index)
            df_de = df_de[[df_index, status_col, *columns[1:]]]
            return df_de
        else: 
            print('\nNo one columns with incorrect date format!\n')
    else:
        date_format = re.compile(r'\d{2}\/\d{2}\/\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                value = df_dates.loc[de_site,de_column]
                if date_format.match(value) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                    else:
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                            
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            df_de.reset_index()
            df_de = pd.merge(df_de, filtered, how='left', on=df_index)
            df_de = df_de[[df_index, status_col, *columns[1:]]]
            return df_de
        else: 
            print('\nNo one columns with incorrect date format!\n')

def check_amounts(df_check, df_index, status_col, columns, pattern=','):
    """
    Paramns:
    pattern: general (. to decimal), other (, to decimal)
    """

    df = df_check[columns]
    df['sites'] = df[df_index]
    df = df.set_index('sites')

    df_new = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df[df_index])
    if not df_duplicates.empty:
        df.drop_duplicates(subset=[df_index], inplace=True)
    
    filtered = df_check[[df_index, status_col]]

    for site in df[df_index]:
        for column in columns[1:]:
            if not str(df.loc[site,column]).__contains__(pattern):
                #print(df.loc[site,column])
                if df.loc[site,df_index] not in df_new.index:
                    df_new.loc[site,df_index] = df.loc[site,df_index]
                    df_new.loc[site,column] = 'Incorret Format to separate decimal values'
                else:
                    df_new.loc[site,column] = 'Incorret Format to separate decimal values'

    df_new = df_new.dropna(how='all', axis=1).fillna('Ok')       
    if not df_new.empty:
        df_new = pd.merge(df_new, filtered, how='left', left_on='sites', right_on=tw_index)
        df_new = df_new[['sites', status_col]]
        return df_new
    else: 
        print('No one columns with incorrect Amount format!')
        
def check_picklist(df,df_index,df_status, df_cols, picklist_dict):

    df_picklist = df[df_cols]
    df_picklist['sites'] = df[df_index]
    df_picklist =  df_picklist.set_index('sites')
    
    #df_picklist = replace_values(df_picklist, df_cols, 0)

    df_errors = pd.DataFrame(columns=df_cols)

    for site in df[df_index]:
        columns = [i.lower() for i in picklist_dict.keys()]
        for column in set(columns): 
            value = str(df_picklist.loc[site,column])
            #print(value)
            if not value in set(picklist_dict[column]) or pd.isnull(value):
                #print(set(picklist_dict[column]))

                if not df_picklist.loc[site,df_index] in df_errors.index:
                    df_errors.loc[site,df_index] = df_picklist.loc[site,df_index]
                    
                    if pd.isnull(value) or pd.isna(value) or value == 'nan' or value == '':
                        df_errors.loc[site,column] = 'Blank Value'
                    else:
                        df_errors.loc[site,column] = f'Incorret picklist value: {value}'
                else:
                    
                    if pd.isnull(value) or pd.isna(value) or value == 'nan' or value == '':
                        df_errors.loc[site,column] = 'Blank Value'
                    else:
                        df_errors.loc[site,column] = f'Incorret picklist value: {value}'

    #df = df_errors.dropna()   
    df_errors = df_errors.dropna(how='all', axis=1).fillna('Ok!')
    if len(df_errors)>0:
        df = df[[df_index, df_status]]
        df_errors = pd.merge(df_errors,df, how='left', on=[df_index])
        df_errors = df_errors.set_index(df_index)
        df_errors = df_errors[[df_status]+ df_errors.columns[:-1].tolist()]
        df_errors = df_errors.reset_index()
    else:
        print('\nNo one Picklist Error Founded!\n')
    return df_errors

def check_picklist_3(df,df_index,df_status,picklist_dict):
    import json
    log = {}

    #picklist_dict={'Categorization by Transmission Sys':['Long-Term Mobile Sites','Macro','Non-Enterprise DAS','Repeater Sites','Public DAS Sites','Transmission']}
    for column in picklist_dict:
        df_aux=df.copy()
        #print(type(picklist_dict[column]))
        new=df_aux[column].isin(picklist_dict[column])
        #print(new)
        indexes=df.index[new == False].tolist()
        if column not in log:log[column]=[]
        log[column]=log[column]+indexes
    newDict = {}
    #print(log)
    #print(json.dumps(picklist_dict,indent=2))
    for key,value in log.items():
      for val in value:
          ID=df.iloc[val][df_index]
          if ID in newDict:
              v=f'Incorret picklist value: {df.iloc[val][key]}'
              if df.iloc[val][key]!=df.iloc[val][key] or pd.isnull(df.iloc[val][key]) or  pd.isna(df.iloc[val][key]) or df.iloc[val][key]=='' or df.iloc[val][key]=='nan':
                   v='Blank Value'
              newDict[ID][key]=v
          else:
              v=f'Incorret picklist value: {df.iloc[val][key]}'
              if df.iloc[val][key]!=df.iloc[val][key] or pd.isnull(df.iloc[val][key]) or  pd.isna(df.iloc[val][key]) or df.iloc[val][key]=='' or df.iloc[val][key]=='nan':
                   v='Blank Value'
              newDict[ID] = {'status': df.iloc[val][df_status],key:v}
    #print(newDict)
    logs = pd.DataFrame.from_dict(newDict, orient='index')
    logs.index.name='Site'
    logs = logs.dropna(how='all', axis=1).fillna('Ok!')
    return logs

def check_new_sites(df_towerdb, tw_index, bts_col, bill_col, status_col, msa_list, towerdb_list, uip_list):
    
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    
    filtered = df_towerdb[[tw_index, status_col]]

    #Finding for ALL NEW SITES
    new_site = [i for i in towerdb_list if i not in msa_list]
    new_sites = pd.DataFrame(new_site, columns=['New_Sites'])
    new_sites = pd.merge(new_sites, filtered, how='left', left_on=['New_Sites'], right_on=tw_index)
    new_sites = new_sites[['New_Sites', status_col]]

    
    #list of new sites out of UIP sheet
    bts_out_uis = [i for i in df_towerdb[df_towerdb[bts_col]=='Yes'][tw_index] if str(i) not in uip_list]
    bts_out_uis = pd.DataFrame(bts_out_uis, columns=['Bts_Sites_Out_UIS_File'])
    bts_out_uis = pd.merge(bts_out_uis, filtered, how='left', left_on=['Bts_Sites_Out_UIS_File'], right_on=tw_index)
    bts_out_uis = bts_out_uis[['Bts_Sites_Out_UIS_File', status_col]]

    #create a copy to make index modifications
    df = df_towerdb.copy()

    #New columns with Site Codes
    df['Sites'] = df[tw_index]
    #defining site code to index
    df.set_index('Sites', inplace=True)

    #filtering by new sites
    df = df[df[tw_index].isin(new_site)]

    # Save information os sites with demerged date more than current date
    df[bill_col] = df[bill_col].astype('datetime64[s]')
    df_site_bts = df[(df[bts_col]=='Yes') | (df[bill_col] > current_date)]
    
    if not (new_sites.empty or bts_out_uis.empty or df_site_bts.empty):
        return new_sites, bts_out_uis, df_site_bts[[tw_index,status_col, bts_col, bill_col]]
        
def check_bts(df_tw, bts_tw_columns, tw_index, status_col, df_msa, bts_msa_column, msa_index):
    #Nested Function to make conditional validations
    def cond_bts_check(bts_msa, tw_bts_sites):
        bts_out_tw=[]
        if sorted(bts_msa) != sorted(tw_bts_sites):
            for i in tw_bts_sites:
                if i not in bts_msa:
                    bts_out_tw.append(i)

        return bts_out_tw

    bts_msa = msa[msa[bts_msa_column]=='Yes']
    bts_msa = [str(i) for i in bts_msa[msa_index]]

    tw_bts_sites = df_tw[df_tw[bts_tw_columns]=='Yes']
    tw_bts_sites = [str(i) for i in tw_bts_sites[tw_index]]

    #return of datas
    filtered = df_tw[[tw_index, status_col]]
    bts_out_tw = cond_bts_check(bts_msa, tw_bts_sites)
    df = pd.DataFrame(bts_out_tw, columns=['New Sites'])
    if not df.empty:
        df = pd.merge(df, filtered, how='left', left_on=['New Sites'], right_on=tw_index)
        df = df[['Bts_Sites_Out_UIS_File', status_col]]
        return df
    else: 
        print('\nNo errors founded!\n')

def check_wip(df_tw,tw_index,tw_status, tw_wip, tw_bts, df_msa, msa_index, wip_msa_col):

    #Nested Function to make conditional validations
    def cond_wip_check(wip_msa, tw_wip_sites):
        count_wip = 0
        wip_out_tw=[]
        if sorted(wip_msa) != sorted(tw_wip_sites):
            for i in tw_wip_sites:
                if i not in wip_msa:
                    count_wip += 1
                    wip_out_tw.append(i)

        return wip_out_tw

    wip_msa = df_msa[df_msa[wip_msa_col]=='Yes']
    wip_msa = [str(i) for i in wip_msa[msa_index]]

    tw_wip_sites = df_tw[df_tw[tw_wip]=='Yes']
    tw_wip_sites = [str(i) for i in tw_wip_sites[tw_index]]

    tw_wip_site_bts_flagged = df_tw[(df_tw[tw_wip]=='Yes')&(df_tw[tw_bts]=='Yes')]
    tw_wip_site_bts_flagged = tw_wip_site_bts_flagged[[tw_index,tw_status,tw_wip, tw_bts]]

    wip_out_tw_list = cond_wip_check(wip_msa, tw_wip_sites)
    if not(len(wip_out_tw_list)==0 or tw_wip_site_bts_flagged.empty):
        return wip_out_tw_list, tw_wip_site_bts_flagged

def check_decommissioned(df,df_index,status_col, decom_col, doer_col):
    #c = country.lower()
    filtered = df[(df[decom_col]=='Yes')&(df[doer_col]=="")]
    if not filtered.empty:
        return filtered[[df_index,status_col, decom_col, doer_col]]
    else:
        print('\nNo errors Founded!\n')
    
def check_tw_bill_doer(df_tw, tw_index, date_col, status_col, status, type_col):
    
    t = type_col.lower()
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    df_tw = df_tw[df_tw[status_col]==status][[tw_index, status_col, date_col]] 
    #print(df_tw)
    if not df_tw[date_col].empty:
        if t == 'doer':
            filtered = df_tw[df_tw[date_col].astype('datetime64[ns]') < current_date]
            return filtered
        else:
            #bill_dates = pd.to_datetime(df_tw[date_col], errors==coerrce)
            filtered = df_tw[df_tw[date_col]=='']
            return filtered
    else:
        print('\nNo errors founded!\n')

def check_tw_doer_planned(df_tw, tw_index, doer_col,bill_col, status_col, dt_format):
    """Only GR until now"""
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date, format=dt_format)
    if not df_tw[doer_col].empty:
        df_tw[bill_col] = pd.to_datetime(df_tw[bill_col],errors='coerce', format=dt_format)
        filtered = df_tw[(df_tw[status_col]=='Planned')&(not df_tw[doer_col].astype('datetime64[ns]').empty)&\
                         (df_tw[bill_col].astype('datetime64[ns]') < current_date)]
        return filtered[[tw_index, status_col, bill_col, doer_col]]  
    else:
        print('\nNo Errors Founded!\n')
                                                              
def check_mom_bts(df_tw, tw_index,status_col, tw_col, df_msa, msa_index, msa_col):

    #c = country   
    msa_bts = df_msa[df_msa[msa_col]=='Yes']
    msa_bts_sites = [i for i in msa_bts[msa_index]]

    tw_bts = df_tw[df_tw[tw_col]=='Yes']
    tw_bts_sites = [i for i in tw_bts[tw_index]]

    out_tower_bts = [i for i in msa_bts_sites if i not in tw_bts_sites]
    filtered = tw_bts[tw_bts[tw_index].isin(out_tower_bts)]
    if not filtered.empty:
        return filtered[[tw_index,status_col, tw_col]]    
    else:
        print('\nNo Errors Founded!\n')

def check_lc_ta_dates(df,tw_index,status_col, start_date,end_date):
    filtered = df[df[start_date] > df[end_date]]
    if not filtered.empty:
        return filtered[[tw_index,status_col, start_date,end_date]]
    else:
        print('\nNo Errors Founded!\n')

def check_uip_tw(df_tw,tw_index, status_tw_col, decom_col, tw_bts_col, tw_critical_col, df_uip, uip_sites, country):
    t1 = ['pt', 'de', 'cz', 'ie', 'es', 'ro', 'hu']

    filtered = df_tw[[tw_index, status_tw_col]]
    if country.lower() in t1:
        #tw_active = df_tw[df_tw['Site Status']=='In Service']
        count_tw_sites = [i for i in df_tw[df_tw[status_tw_col] =='In Service'][tw_index]]

        # check number of sites that are in uip file and doesn't have in df_tw
        in_service_uip_sites = []
        if not set(count_tw_sites).intersection(uip_sites):
            in_service_uip_sites = [i for i in uip_sites if i not in count_tw_sites]
        in_service_uip_sites = pd.DataFrame(in_service_uip_sites,columns=['Site In service out of UIS File!'])
        in_service_uip_sites = pd.merge(in_service_uip_sites, filtered, how='left', left_on='Site In service out of UIS File!',\
                                        right_on=tw_index)
        in_service_uip_sites = in_service_uip_sites[['Site In service out of UIS File!', status_tw_col]]
        
        #check for decomissioned site not in uip files
        if decom_col != "":
            tw_decomiss = [i for i in df_tw[df_tw[decom_col]=='Yes'][tw_index] if i in uip_sites]
            decomiss_sites_in_uip = pd.DataFrame(tw_decomiss, columns=['Decomissioned Site in UIS File'])
            decomiss_sites_in_uip = pd.merge(decomiss_sites_in_uip, filtered, how='left', left_on='Decomissioned Site in UIS File',\
                                         right_on=tw_index)
            decomiss_sites_in_uip = decomiss_sites_in_uip[['Decomissioned Site in UIS File', status_tw_col]]
        
            #Check BTS sites
            bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
            bts_sites_out_uip = []
            if not set(bts_sites).intersection(uip_sites):
                bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
            bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site not in UIS File'])
            bts_sites_out_uip = pd.merge(bts_sites_out_uip, filtered, how='left', left_on='BTS Site not in UIS File',\
                                         right_on=tw_index)
            bts_sites_out_uip = bts_sites_out_uip[['BTS Site not in UIS File', status_tw_col]]

            #  Check for UIP critical sites 
            uip_critical = [i for i in df_uip[(df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!='')&\
                                        df_uip['BTS site applicable charge (Annual)']!=""]['Site_ID']]
            bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]
            critical = []
            if len(uip_critical) > 0:
                if set(uip_critical).intersection(bts_tw_critical):
                    critical = [i for i in bts_tw_critical if i not in uip_critical]
            critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% in out UIS File'])
            critical = pd.merge(critical, filtered, how='left', left_on='Sites with critical level beyond 10% in out UIS File',\
                                         right_on=tw_index)
            critical = critical[['Sites with critical level beyond 10% in out UIPS File', status_tw_col]]
            if not (in_service_uip_sites.empty or decomiss_sites_in_uip.empty or bts_sites_out_uip.empty or critical.empty):
                return in_service_uip_sites, decomiss_sites_in_uip, bts_sites_out_uip, critical
        else:
            #Check BTS sites
            bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
            bts_sites_out_uip = []
            if not set(bts_sites).intersection(uip_sites):
                bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
            bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site not in UIS File'])
            bts_sites_out_uip = pd.merge(bts_sites_out_uip, filtered, how='left', left_on='BTS Site not in UIS File',\
                                         right_on=tw_index)
            bts_sites_out_uip = bts_sites_out_uip[['BTS Site not in UIS File', status_tw_col]]

            #  Check for UIP critical sites 
            uip_critical = [i for i in df_uip[(df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!='')&\
                                        df_uip['BTS site applicable charge (Annual)']!=""]['Site_ID']]
            bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]
            critical = []
            if len(uip_critical) > 0:
                if set(uip_critical).intersection(bts_tw_critical):
                    critical = [i for i in bts_tw_critical if i not in uip_critical]
            critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% in out UIS File'])
            critical = pd.merge(critical, filtered, how='left', left_on='Sites with critical level beyond 10% in out UIS File',\
                                         right_on=tw_index)
            critical = critical[['Sites with critical level beyond 10% in out UIS File', status_tw_col]]
            if not (in_service_uip_sites.empty or bts_sites_out_uip.empty or critical.empty):
                return in_service_uip_sites, bts_sites_out_uip, critical
    else:
        #tw_active = df_tw[df_tw['Site Status']=='In Service']
        count_tw_sites = [i for i in df_tw[df_tw[status_tw_col] =='In Service'][tw_index]]

        # check number of sites that are in uip file and doesn't have in df_tw
        in_service_uip_sites = []
        if not set(count_tw_sites).intersection(uip_sites):
            in_service_uip_sites = [i for i in uip_sites if i not in count_tw_sites]
        in_service_uip_sites = pd.DataFrame(in_service_uip_sites,columns=['Site In service out of UIS File!'])
        in_service_uip_sites = pd.merge(in_service_uip_sites, filtered, how='left', left_on='Site In service out of UIS File!',\
                                        right_on=tw_index)
        in_service_uip_sites = in_service_uip_sites[['Site In service out of UIS File!', status_tw_col]]
        
        #check for decomissioned site not in uip files
        tw_decomiss = [i for i in df_tw[df_tw[decom_col]=='Yes'][tw_index]]
        decomiss_sites_in_uip = []
        if set(tw_decomiss).intersection(uip_sites):
            decomiss_sites_in_uip = [i for i in tw_decomiss if i in uip_sites]
        decomiss_sites_in_uip = pd.DataFrame(decomiss_sites_in_uip, columns=['Decomissioned Site in UIS File'])
        decomiss_sites_in_uip = pd.merge(decomiss_sites_in_uip, filtered, how='left', left_on='Decomissioned Site in UIS File',\
                                         right_on=tw_index)
        decomiss_sites_in_uip = decomiss_sites_in_uip[['Decomissioned Site in UIS File', status_tw_col]]
        
        #Check BTS sites
        bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
        bts_sites_out_uip = []
        if not set(bts_sites).intersection(uip_sites):
            bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
        bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site not in UIS File'])
        bts_sites_out_uip = pd.merge(bts_sites_out_uip, filtered, how='left', left_on='BTS Site not in UIS File',\
                                         right_on=tw_index)
        bts_sites_out_uip = bts_sites_out_uip[['BTS Site not in UIS File', status_tw_col]]

        #  Check for UIP critical sites 
        uip = [i for i in df_uip['Site_ID']]
        bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Yes'][tw_index]
        critical = []
        if set(uip).intersection(bts_tw_critical):
            critical = [i for i in bts_tw_critical if i not in uip]
        critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% out in UIS File'])
        critical = pd.merge(critical, filtered, how='left', left_on='Sites with critical level beyond 10% in out UIS File',\
                                         right_on=tw_index)
        critical = critical[['Sites with critical level beyond 10% in out UIS File', status_tw_col]]
        if not (in_service_uip_sites.empty or decomiss_sites_in_uip.empty or bts_sites_out_uip.empty or critical.empty):
            return in_service_uip_sites, decomiss_sites_in_uip, bts_sites_out_uip, critical

def check_commercial(path_current, path_before, col_replace, col_names, merge_cols, col_order):
    
    def replace_values(df, columns, value=""):
        """
        Está voltando para float
        """
        invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan']
        #lista = []

        df.fillna("", inplace=True)
        #df[column] = df[column].astype('int64')

        for column in columns:
            lista = []
            for index in df[column]:
                #print(f"{column} -> {index}")
                if index in invalid_values:
                    lista.append(value)
                else:
                    lista.append(index)
            df[column] = lista

        return df    
    
    def check_uip_commercial_values(df_actual, df_before, merge_cols):
        df_atual = uip_comercial_actual
        df_ant = uip_comercial_before
        df_cross = pd.merge(df_actual, df_before, on=merge_cols, \
                            how='left', suffixes=('_actual', '_before'), indicator='Exist')
        df_cross['Exist'] = np.where(df_cross.Exist == 'both', 'Yes', 'No')
        df_cross['Equal Values'] = df_cross['Exist']
        
        return df_cross
    
    # Check for commercial Values into current UIP File and compare with UIP File before
    uip_comercial_actual = pd.read_excel(path_current,sheet_name='Commercial', names=col_names).fillna('')
    #uip_comercial_actual = uip_comercial_actual[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_actual = replace_values(uip_comercial_actual, col_replace, value=0)

    uip_comercial_before = pd.read_excel(path_before,sheet_name='Commercial', names=col_names ).fillna('')
    #uip_comercial_before = uip_comercial_before[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_before = replace_values(uip_comercial_before, col_replace, value=0)

    df_commercial =  check_uip_commercial_values(uip_comercial_actual, uip_comercial_before, merge_cols).fillna('')

    df_commercial = df_commercial.reindex(columns=col_order)
    df_commercial_diffs = df_commercial[df_commercial['Equal Values']=='No']
    if not df_commercial_diffs.empty:
        return df_commercial_diffs
    else:
        print('\nNo errors founded!')

def check_diffs_v2(path_current, path_last, cols_order, type_file='Excel', sheet='Commercial'):
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def highlight_diff(data, color='yellow'):
        attr = 'background-color: {}'.format(color)
        other = data.xs('Current', axis='columns', level=-1)
        return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                            index=data.index, columns=data.columns)
    type_file = type_file.lower()
    if type_file =='excel':
        _actual = pd.read_excel(path_current,sheet_name=sheet).fillna('')

        _before = pd.read_excel(path_last,sheet_name=sheet).fillna('')

        df_all = pd.concat([_actual, _before],axis='columns', keys=['Current', 'Last'])
        df_final = df_all.swaplevel(axis='columns')[_actual.columns[1:]]

        #df_final.style.apply(highlight_diff, axis=None)
        if not df_final.empty:
            return df_final[(_actual != _before).any(1)].style.apply(highlight_diff, axis=None)
        else:
            print('\nNo differences Founded!\n')

def general_log_erros(df_list, sheet_list, path):
    writer = pd.ExcelWriter(path,engine='openpyxl')   
    for dataframe, sheet in zip(df_list, sheet_list):
        dataframe.to_excel(writer, sheet_name=sheet, startrow=0 , startcol=0)   
    writer.save() 

def find_diffs_between_files(path_OLD, path_NEW, index_col, bill_cols, \
                             status_col, path_save, old_name, new_name, type_file='mix',kind='tw', sheetname='', skipr=0, skipc=0):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, bill_cols):
        df.columns = lower_str(list(df.columns))
        df = df[bill_cols]
        df = df.dropna(subset=[index_col], axis=0)
        df = df.drop_duplicates(subset=[index_col], keep='last')
        df[index_col] = df[index_col].astype(str)
        df['sites'] = df[index_col]
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,bill_cols)

    elif type_file == 'csv':
        df_OLD = pd.read_csv(path_OLD,engine='python',encoding='latin1').fillna('')
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_csv(path_NEW,engine='python').fillna('')
        df_NEW = fit_df(df_NEW,bill_cols)

    elif type_file == 'mix':
        df_OLD = pd.read_csv(path_OLD, encoding='latin')
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,bill_cols)
    else: 
        df_OLD = fit_df(path_OLD,bill_cols)
        df_NEW = fit_df(path_NEW,bill_cols)


    # Perform Diff
    new_copy = df_NEW.copy()
    droppedRows = []
    newRows = []

    cols_OLD = df_OLD.columns
    cols_NEW = df_NEW.columns
    sharedCols = list(set(cols_OLD).intersection(cols_NEW))

    for row in new_copy.index:
        if (row in df_OLD.index) and (row in df_NEW.index):
            for col in sharedCols:
                value_OLD = str(df_OLD.loc[row,col])
                value_NEW = str(df_NEW.loc[row,col])
               #Error de a.empty() são sites duplcados e nã poodem estar no index
                if value_OLD == value_NEW:
                    new_copy.loc[row,col] = np.nan
                else:
                    new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
        else:
            newRows.append(row)

    new_copy = new_copy.dropna(axis=0, how='all')
    new_copy = new_copy.dropna(axis=1, how='all')

    for row in df_OLD.index:
        if row not in df_NEW.index:
            droppedRows.append(row)
            new_copy = new_copy.append(df_OLD.loc[row,:])
    
    new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
    
    new_copy = new_copy.reset_index()
    if kind=='tw':
        sites = [i for i in new_copy['sites']] 
        old = df_OLD[[status_col]].reset_index()
        old = old.loc[old['sites'].isin(sites)]
        new = df_NEW[[status_col]].reset_index()
        new = new.loc[new['sites'].isin(sites)]
        df_cross = pd.merge(new, old, on=['sites'], how='inner', suffixes=('_current', '_before'))
        new_copy = pd.merge(new_copy, df_cross, on=['sites'], how='left')
        status_1 = f'{status_col}_current'
        status_2 = f'{status_col}_before'
        new_copy = new_copy.set_index('sites')
        new_copy = new_copy[[status_1, status_2]+ new_copy.columns[:-2].tolist()]
        new_copy = new_copy.reset_index()


    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save} - ({old_name}) vs ({new_name}).xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    new_copy.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_header = PatternFill(start_color='00FF0000', \
                                end_color='00FF0000', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ10000', highlight)
    
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ10000', header_style)
    
    #print(len(newRows))
    for site in new_copy['sites']:
        if site in newRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)
    
    wb.save(fname)
    print('\nDone.\n')

def find_diffs_between_files_1(path_OLD, path_NEW, index_col, bill_cols, \
                             path_save, old_name,new_name, type_file='mix', dates_cols=[], status_col='', kind='tw',kind_col='', sheetname='', skipr=0, skipc=0):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, index_col, kind, kind_col):
        kind_col = kind_col.lower()
        kind = kind.lower()
        if kind == 'ta':
            df = df.dropna(subset=[index_col], axis=0)
            df['sites'] = df[index_col].astype(str) + df[kind_col] 
            #df.columns = lower_str(list(df.columns))
            df = df.dropna(subset=['sites'], axis=0)
            # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
            df = df.set_index('sites').fillna('')
        else:
            df = df.dropna(subset=[index_col], axis=0)
            df = df.drop_duplicates(subset=[index_col], keep='last')
            df[index_col] = df[index_col].astype(str)
            df['sites'] = df[index_col]
            # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
            df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    def csv_header(path):
        """Função usada para ler os ficheiros que tem o simbolo do Euro"""
        import csv
        f = open(path, encoding='windows-1252', errors='ignore')
        data = []
        for row in csv.reader(f, delimiter=','):
            data.append(row)
        col = lower_str([*data[0]])
        #data.pop(0)
        #df = pd.DataFrame(data, columns=col)
        return  col

    def comparison(df_old, df_new,status):
        # Perform Diff
        new_copy = df_NEW.copy()
        droppedRows = []
        newRows = []

        cols_OLD = list(df_OLD.columns)
        cols_NEW = list(df_NEW.columns)
        sharedCols = list(set(cols_OLD).intersection(cols_NEW))
        #print(sharedCols)
        for row in new_copy.index:
            if (row in df_OLD.index) and (row in df_NEW.index):
                for col in sharedCols:
                    value_OLD = str(df_OLD.loc[row,col])
                    value_NEW = str(df_NEW.loc[row,col])
                #Error de a.empty() são sites duplcados e nã poodem estar no index
                    if value_OLD == value_NEW:
                        new_copy.loc[row,col] = np.nan
                    else:
                        new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
            else:
                newRows.append(row)

        new_copy = new_copy.dropna(axis=0, how='all')
        new_copy = new_copy.dropna(axis=1, how='all')

        for row in df_OLD.index:
            if row not in df_NEW.index:
                droppedRows.append(row)
                new_copy = new_copy.append(df_OLD.loc[row,:])
        
        new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
        
        new_copy = new_copy.reset_index()

        if kind=='tw':
            sites = [i for i in new_copy['sites']] 
            old = df_OLD[[status]].reset_index()
            old = old.loc[old['sites'].isin(sites)]
            new = df_NEW[[status]].reset_index()
            new = new.loc[new['sites'].isin(sites)]
            df_cross = pd.merge(new, old, on=['sites'], how='inner', suffixes=('_current', '_before'))
            new_copy = pd.merge(new_copy, df_cross, on=['sites'], how='left')
            status_1 = f'{status}_current'
            status_2 = f'{status}_before'
            new_copy = new_copy.set_index('sites')
            new_copy = new_copy[[status_1, status_2]+ new_copy.columns[:-2].tolist()]
            new_copy = new_copy.reset_index()

        return newRows, droppedRows, new_copy

    def date_parser(df, columns, format=1, type_dates='normal'):
        t_col = type_dates.lower()
        if format == 1:
            type_date = "%d/%m/%Y"
        else:
            type_date = "%d-%m-%Y"
        for column in lower_str(columns):
            if t_col == 'mixed':
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) \
                            and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
                df[column] = df[column].astype(str)
            else:
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD.columns = lower_str(list(df_OLD.columns)) 
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,index_col,kind, kind_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,index_col,kind, kind_col)

    elif type_file == 'csv':
        #cols_old = csv_header(path_OLD)
        df_OLD = pd.read_csv(path_OLD,engine='python', encoding='latin1').fillna('')
        df_OLD.columns = lower_str(list(df_OLD.columns))
        df_OLD = df_OLD[bill_cols]
        df_OLD = fit_df(df_OLD, index_col, kind, kind_col)

        #cols_new = csv_header(path_NEW)
        #Se for o arquivo gerado a partir do xlsx não prcisa de encoding
        df_NEW = pd.read_csv(path_NEW, engine='python', encoding='latin1').fillna('')
        df_NEW.columns = lower_str(list(df_NEW.columns))
        df_NEW = df_NEW[bill_cols]
        df_NEW = fit_df(df_NEW, index_col,kind, kind_col)

    else:
        #cols_old = csv_header(path_OLD) header=0, names=cols_old,
        df_OLD = pd.read_csv(path_OLD, engine='python').fillna('')
        df_OLD.columns = lower_str(list(df_OLD.columns))
        df_OLD = fit_df(df_OLD, index_col, kind, kind_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        date_parser(df_NEW, dates_cols, 1, 'normal')
        df_NEW = fit_df(df_NEW, index_col, kind, kind_col)

    newRows, droppedRows, df_all = comparison(df_OLD, df_NEW, status_col)

    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save} - ({old_name}) vs ({new_name}).xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    df_all.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_header = PatternFill(start_color='00FF0000', \
                                end_color='00FF0000', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ10000', highlight)
    
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ10000', header_style)
    
    #print(len(newRows))
    for site in df_all['sites']:
        if site in newRows:
            idx = df_all.index[df_all[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = df_all.index[df_all[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)

    wb.save(fname)
    print('\nDone.\n')

Looking for differences betwwnn older and newest files in towerdb.

Tem colunas com mesmo nome e está gerando erros na hora de correr o script por toda a planilha.

In [ ]:
hu_bill_cols = ["Active / not active",\
    "Confirmed Skylon scope",\
    "Code",\
    "Categorization by Transmission Sys",\
    "Additional information \x96 room configuration (including container information)",\
    "Active Sharing Arrangements involving the Operator",\
    "Critical site",\
    "Strategic site",\
    "Telenor tenant",\
    "Telekom tenant",\
    "Power supply for tenants",\
    "Energy category",\
    'Ready For Active Installation ("RFAI") date',\
    "Wip_Site",\
    "Bts_Site",\
    "Strategic_Site_Bucket",\
    "Critical_Site_Beyond_10",\
    "Subsequent_Sharing_Arrangement",\
    "First_Active_Sharing_Start_Date",\
    "First_Active_Sharing_End_Date",\
    "Active Sharing Deployment Types",\
    "Sites_As_Metered_Estimated",\
    "Date of equipment removal"]

path_tw = '/content/TowerDB_Hungary_20210731.csv'
path_msa = '/content/TowerDB_Hungary_20210630.csv'   
tw_save = '/content/TW_HU'
new_name = 'TowerDB_Hungary_20210731.csv'
old_name = 'TowerDB_Hungary_20210630.csv'  
tw_dates = ['ready for active installation ("rfai") date',"First_Active_Sharing_Start_Date",\
            "First_Active_Sharing_End_Date", "Date of equipment removal"]
find_diffs_between_files_1(path_msa, path_tw, 'Code',hu_bill_cols, tw_save, old_name, new_name, type_file='csv',status_col='active / not active',kind='tw')

Doing all checks in Towerdb File

In [ ]:
path_msa = '/content/TowerDB_Hungary_20210630.csv'
msa = pd.read_csv(path_msa, encoding='latin')

msa_cols = lower_str(list(msa.columns))
msa.columns = msa_cols

path_tw = '/content/TowerDB_Hungary_20210731.csv'
towerdb = pd.read_csv(path_tw, encoding='latin').fillna('')
towerdb.columns = lower_str(list(towerdb.columns))

"""Check Columns Received"""             
df_cols = check_columns_received(towerdb, msa_cols)
df_cols
#No columns missing

,Column(s) Missing
0,"vertical infrastructure \n(ex: torre, suportes..."
1,site type\n(ex. rtt; gbt)
2,"technology_vod\n(ex: umts,gsm,lte)"
3,fibre/microwave\n(fo/mw)
4,"power supply\n(ex: ac power, dc power)"


In [ ]:
"""Defining variables which is gonna be reusable in checks"""
tw_index = 'code'
tw_doer = 'date of equipment removal'
tw_status = 'active / not active'
tw_bts = 'bts_site'
tw_bill = 'ready for active installation ("rfai") date'
tw_wip = 'wip_site'
#tw_decom = 'Decommissioned sites'
tw_critical = 'critical site'


msa_index ='code'
msa_doer = 'date of equipment removal'
msa_status = 'active / not active'
msa_bts = 'bts_site'
msa_bill = 'ready for active installation ("rfai") date'
msa_wip = 'wip_site'
#msa_decom = 'Decommissioned sites'
msa_critical = 'critical site'

First Check - Dates Formats (dd/mm/YYYY)

Columns: Date of equipment removal

In [1]:
"""You need to convert all values in cols for string format to check"""
"""CSV FIles doesn't need convert, only fill na values"""
# Columns to functions
dates_doer = [tw_index, tw_doer]
dates_bill = [tw_index, tw_bill]
#Columns to parser
doer=[tw_doer]
bill=[tw_bill]


#date_parser(towerdb, bill, "%d/%m/%Y", 'no')
#date_parser(towerdb, doer, "%d/%m/%Y", 'mix')

actives_1 = towerdb[towerdb[tw_status]=='In Service']
no_actives_1 = towerdb[~(towerdb[tw_status]=='In Service')]

#Checking columns for errors
actives_dates_errors = check_date_columns(actives_1, tw_index,tw_status, dates_bill, 2) 
# Actives sites with blank billing trigger date
no_actives_dates_errors = check_date_columns(no_actives_1, tw_index, tw_status, dates_doer, 2) 
no_actives_dates_errors

NameError: ignored

In [ ]:
#Checking columns for errors
actives_dates_errors


,code,active / not active,"ready for active installation (""rfai"") date"
0,1001,In Service,Blank Value
1,1002,In Service,Blank Value
2,1004,In Service,Blank Value
3,1005,In Service,Blank Value
4,1007,In Service,Blank Value
...,...,...,...
2017,RGU0349,In Service,Blank Value
2018,RGU0352,In Service,Blank Value
2019,RGU0453,In Service,Blank Value
2020,RU0094,In Service,Blank Value


Second Check - TW Amount value General (xxx.xx)

Column(s): ???

In [ ]:
amount_cols = [tw_index, """???"""]
df_amount_errors = check_amounts(actives, tw_index, amount_cols, 1)
#No one error

Thirth - Check Picklist values All sites

Do this check in all sites

Check the picklist for each case

Check - Remove "N/A", "0" or "-" values

In [ ]:
bill_cols = ['confirmed skylon scope','code','categorization by transmission sys', 'categorization by site type',\
             'additional information  room configuration (including container information)', 'type of air cooling', \
             'active sharing arrangements involving the operator', 'critical site','strategic site','telenor tenant','telekom tenant',\
             'power supply for tenants','energy category', 'ready for active installation ("rfai") date', 'wip_site','bts_site','strategic_site_bucket',\
             'critical_site_beyond_10','subsequent_sharing_arrangement','first_active_sharing_start_date','first_active_sharing_end_date',\
             'active sharing deployment types','sites_as_metered_estimated']
towerdb = replace_values(towerdb, bill_cols)

Thirth - Check Picklist values All sites

Do this check in all sites

Check the picklist for each case

In [ ]:
picklist_tw_general = {
    'categorization by transmission sys': ['Macro', 'Macro+DAS', 'Public DAS','Transmission']
}
pick_col_general = ['code', 'categorization by transmission sys']

df_general_pick = check_picklist(towerdb, tw_index,tw_status, pick_col_general, picklist_tw_general)
df_general_pick


No one Picklist Error Founded!



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:272: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


""


Check Picklist and dates formats for In service sites



In [ ]:
picklis_dict = {
    'categorization by transmission sys': ['Macro', 'Macro+DAS', 'Public DAS','Transmission'],
    'categorization by site type': ['DAS passive','GBT','RTT', 'RTT+DAS passive'],
    'type of air cooling' : ['Yes', 'No'],
    'critical site': ['Yes', 'No'],
    'strategic site': ['Yes', 'No'],
    'power supply for tenants': ['AC', 'DC', 'no power'],
    'bts_site': ['Yes', 'No'],
    'strategic site': ['Yes', 'No'],
    'critical site': ['Yes', 'No'],
    'strategic_site_bucket': ['Non Strategic'],
    'critical_site_beyond_10':['Within 10%','Non Critical'],
    'wip_site': ['Yes', 'No'],
    'sites_as_metered_estimated': ['Estimated Model','Metered Model']
}

picklist_cols = ['code','categorization by transmission sys', 'categorization by site type', 'type of air cooling',\
                 'critical site','strategic site','power supply for tenants', 'wip_site','bts_site',\
                 'strategic_site_bucket','critical_site_beyond_10','sites_as_metered_estimated','strategic site','critical site']

actives_2 = towerdb[towerdb[tw_status]=='In Service']
df_in_service_picklist = check_picklist_3(actives_2,tw_index,tw_status, picklis_dict)
 #= check_picklist(actives, tw_index,tw_status, picklist_cols, picklis_dict)
df_in_service_picklist
#muitos errors


In [ ]:
def check_date_columns(df, df_index,status_col,columns, format):
    """
    Paramns \n
        Dates needs to be in string format not in datetime, otherwise raise an error.\n
        Convert entire column to string before.
    """
    df_dates = df[columns]
    df_dates['sites'] = df_dates[df_index]
    df_dates = df_dates.set_index('sites')

    df_de = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df_dates[df_index])
    if not df_duplicates.empty:
        df_dates.drop_duplicates(subset=[df_index], inplace=True)

    filtered = df[[df_index, status_col]]

    if format == 1:
        date_format = re.compile(r'\d{2}\-\d{2}\-\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                #print(type(df_dates.loc[de_site,de_column]))
                value = df_dates.loc[de_site,de_column]
                if date_format.match(value) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                    else:
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            df_de.reset_index()
            df_de = pd.merge(df_de, filtered, how='left', on=df_index)
            df_de = df_de[[df_index, status_col, *columns[1:]]]
            return df_de
        else: 
            print('\nNo one columns with incorrect date format!\n')
    else:
        date_format = re.compile(r'\d{2}\/\d{2}\/\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                value = df_dates.loc[de_site,de_column]
                if date_format.match(value) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                    else:
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                            
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            df_de.reset_index()
            df_de = pd.merge(df_de, filtered, how='left', on=df_index)
            #df_de = df_de[[df_index, status_col, *columns[1:]]]
            return df_de
        else: 
            print('\nNo one columns with incorrect date format!\n')

actives_1 = towerdb[towerdb[tw_status]=='In Service']
date_cols = ['code','infrastructure ready (existing)/ to be ready (new)', 'ready for active installation ("rfai") date', 'first_active_sharing_start_date','first_active_sharing_end_date']
start_dates = ['infrastructure ready (existing)/ to be ready (new)', 'ready for active installation ("rfai") date', 'first_active_sharing_start_date','first_active_sharing_end_date']

#date_parser(actives, start_dates, "%d/%m/%Y", 'no')
df_in_service_dates = check_date_columns(actives_1, tw_index,tw_status,date_cols, 2)
df_in_service_dates
#Muitos Campos em branco

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,code,"ready for active installation (""rfai"") date",first_active_sharing_start_date,first_active_sharing_end_date,active / not active
0,1001,Blank Value,Blank Value,Blank Value,In Service
1,1002,Blank Value,Blank Value,Blank Value,In Service
2,1004,Blank Value,Blank Value,Blank Value,In Service
3,1005,Blank Value,Blank Value,Blank Value,In Service
4,1007,Blank Value,Blank Value,Blank Value,In Service
...,...,...,...,...,...
2043,RGU0352,Blank Value,Blank Value,Blank Value,In Service
2044,RGU0453,Blank Value,Blank Value,Blank Value,In Service
2045,RU0094,Blank Value,Blank Value,Blank Value,In Service
2046,RU0138,Blank Value,Blank Value,Blank Value,In Service


Fifth Check BTS Flagged(Billing Trigger and Commercial)

In [ ]:
status = 'Yes'
df_bts_flagged = check_tw_bill_doer(actives, tw_index,tw_bill, tw_bts, status, 'bill')
df_bts_flagged
#No errors

Check MoM Sites (BTS)

In [ ]:
""" BTS sites"""
path_uip = '/content/UserInput_Hungary_20210731.xlsx'
uip_names = ['Site_ID','BTS site applicable charge (Annual)',\
             'Commercials for sites beyond 10% cap of critical sites (Annual)']
uip = pd.read_excel(path_uip ,sheet_name='SiteLevel',usecols=[0,1,2],skiprows=2)
uip.columns = uip_names

msa_sites = [str(i) for i in msa[msa_index]]
tw_sites = [str(i) for i in towerdb[tw_index]]
uip_sites = [str(i) for i in uip['Site_ID']]

In [ ]:
df_mom_bts = check_mom_bts(actives, tw_index, tw_bts, msa, msa_index, msa_bts)
df_mom_bts
# No one error df_mom_bts

Check UIP sites with towerDB
Some countries doesn't have decommissioned column

In [ ]:
in_service_uip_sites, bts_sites_out_uip, critical = check_uip_tw(towerdb,tw_index, tw_status, '', tw_bts,tw_critical,\
                                                                 uip, uip_sites, 'hu')
#no errors

Commercial

In [ ]:
path_before = '/content/UserInput_Hungary_20210630.xlsx'
names = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2', 'Data_Type', 'Input_Value',\
        'Description/Instruction', 'Frequency of Update']
merge_cols = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2', 'Data_Type', \
              'Description/Instruction', 'Frequency of Update']
cols_ordered = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2','Data_Type','Input_Value_actual',\
                'Input_Value_before','Equal Values','Description/Instruction','Frequency of Update']
col_replaced = ['Input_Value']
df_commercial_diffs = check_commercial(path_uip, path_before,col_replaced, names, merge_cols, cols_ordered)
#df_commercial_diffs = check_commercial(path_uip, path_before, replace_values, names, merge_cols, cols_ordered)
df_commercial_diffs

Excel Log

In [ ]:
df_list = [df_in_service_dates, df_in_service_picklist]
sheetnames = ['In services dates errrors', 'In servive picklist errors']

path = '/content/HU_towerdb_errors.xlsx'
general_log_erros(df_list, sheetnames, path)

In [ ]:
towerdb.loc[towerdb[tw_index]=='1001'][['ready for active installation ("rfai") date', 'first_active_sharing_start_date','first_active_sharing_end_date']]


TA Input

In [ ]:
pathta = '/content/TA_Input_Hungary_20210731.csv'

ta = pd.read_csv(pathta,  encoding='latin')

ta_leases = ['Base fee (@ signing) in HUF']
ta = replace_values(ta,ta_leases, '')

ta_cols = ['SiteID', 'Base fee (@ signing) in HUF']  
df_ta_amount = check_amounts(ta, 'SiteID', ta_cols)
print(df_ta_amount)
# No errors

df_ta_dates = check_lc_ta_dates(ta,'SiteID', 'Starting date', 'Expiring date (raw data)')
#No erros

Looking for differences between newest and oldest TA file.

In [ ]:
pathta = '/content/TA_Input_Hungary_20210731.csv'
pathold = '/content/TA_Input_Hungary_20210630.csv'
ta_save = '/content/HU_TA'
ta_bill = ['SiteID', 'MNO classification','Base fee (@ signing) in HUF', 'Starting date', 'Expiring date (raw data)']
find_diffs_between_files(pathold, pathta, 'SiteID', ta_bill,'', ta_save,'csv', 'ta')

LC Input

In [ ]:
ta = pd.read_csv(pathta,  encoding='latin')
ta.columns

In [ ]:
 #separador virgulas
pathlc = '/content/LC_Input_Hungary_20210731.csv'

lc = pd.read_csv(pathlc,  encoding='latin')
leases = ['Current annual lease fees in HUF']
lc = replace_values(lc,leases, '')

lc_cols = ['Code', 'Current annual lease fees in HUF']  

df_lc_amount = check_amounts(lc, 'Code', lc_cols)
# No errors

df_lc_dates = check_lc_ta_dates(lc,'Code', 'Starting date', 'Expiring date (raw data)')
#No erros

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:217: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
lc.columns

In [ ]:
pathlc = '/content/LC_Input_Hungary_20210731.csv'
patholdlc = '/content/LC_Input_Hungary_20210630.csv'
lc_save = '/content/HU_LC'
lc_bill = ['Code', 'Current annual lease fees in HUF','Starting date', 'Expiring date (raw data)']
find_diffs_between_files(patholdlc, pathlc, 'Code', lc_bill,'', lc_save,'csv', 'ta')


New Rows:     ['5028', 'S0113']
Dropped Rows: []

Done.



In [ ]:
p = re.compile(r'^\d+(\.\d*[1-9])?$')
v = '10200,01'
if not str(v).__contains__(','):
#if p.match(v) == None:
    print("Erro")
else:
    print(v)

10200,01
